In [22]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [23]:
import pandas as pd
import numpy as np

stores_df = pd.read_excel("https://data.gov.au/data/dataset/2fe5e2a9-8a3d-4dcf-baec-c5147d953150/resource/b59a15df-86ea-4c4c-95be-4dd9fc9f8ac4/download/2019-20-historical-migration-statistics-locked.xlsx", sheet_name="1.2", header=7 )
# stores_df = pd.read_excel("datasets/2019-20-historical-migration-statistics-locked.xlsx", sheet_name="1.2", header=7 )

stores_df = stores_df.iloc[: , :-1]
stores_df = stores_df.iloc[: , 1:]
stores_df = stores_df.melt(id_vars=["Year"], 
        var_name="country", 
        value_name="migration_no")
stores_df.rename(index=str, columns={"Year": "year"}, inplace=True)

stores_df.head()


,year,country,migration_no
0,Jan-Jun 1959,Australia,241.0
1,1959–60,Australia,674.0
2,1960–61,Australia,735.0
3,1961–62,Australia,745.0
4,1962–63,Australia,739.0


In [24]:
stores_df.dropna(subset=['migration_no', 'country' ], inplace=True) 

stores_df['year'].replace(to_replace=".*([0-9]{4}).*", value=r"\1", regex=True, inplace=True)
stores_df['migration_no'].replace(to_replace="\D*(\d*)", value=r"\1", regex=True, inplace=True)

stores_df[['year','migration_no']] = stores_df[['year','migration_no']].astype(int)

stores_df = stores_df[~stores_df.country.str.contains('|'.join(['Australia', 'Other', 'total']))]

stores_df.dtypes


,year,country,migration_no
1740,1959,Canada,126
1741,1959,Canada,354
1742,1960,Canada,602
1743,1961,Canada,589
1744,1962,Canada,641
1745,1963,Canada,614
1746,1964,Canada,824
1747,1965,Canada,865
1748,1966,Canada,795
1749,1967,Canada,920


In [25]:
stores_df['country'].replace(["U.S.S.R."], "Russia",  inplace=True)
stores_df['country'].replace( ["United Kingdom and Ireland"], "United Kingdom",  inplace=True)

import pycountry
list_countries = stores_df['country'].unique().tolist()
d_country_code = {} 
for country in list_countries:
    try:
        country_data = pycountry.countries.search_fuzzy(country)
        country_code = country_data[0].alpha_3
        d_country_code.update({country: country_code})
    except:
        print('\n could not add ISO 3 code for:', country)
        d_country_code.update({country: np.nan})
print(d_country_code)


 could not add ISO 3 code for: Czechoslovakia

 could not add ISO 3 code for: Yugoslavia

 could not add ISO 3 code for: At Sea & Not Stated
{'Fiji': 'FJI', 'New Zealand': 'NZL', 'Papua New Guinea': 'PNG', 'Albania ': 'ALB', 'Austria': 'AUT', 'Belgium': 'BEL', 'Bulgaria ': 'BGR', 'Czechoslovakia': nan, 'Denmark': 'DNK', 'Estonia ': 'EST', 'Finland': 'FIN', 'France': 'FRA', 'Germany': 'DEU', 'Greece': 'GRC', 'Hungary': 'HUN', 'Italy': 'ITA', 'Latvia ': 'LVA', 'Lithuania ': 'LTU', 'Malta': 'MLT', 'Netherlands': 'NLD', 'Norway': 'NOR', 'Poland': 'POL', 'Portugal': 'PRT', 'Romania': 'ROU', 'Spain': 'ESP', 'Sweden': 'SWE', 'Switzerland': 'CHE', 'Russia': 'RUS', 'Ukraine ': 'UKR', 'United Kingdom': 'GBR', 'Yugoslavia': nan, 'Bangladesh ': 'BGD', 'Myanmar': 'MMR', 'China': 'CHN', 'Cyprus': 'CYP', 'Hong Kong': 'HKG', 'India': 'IND', 'Indonesia': 'IDN', 'Israel': 'ISR', 'Japan': 'JPN', 'Lebanon': 'LBN', 'Malaysia': 'MYS', 'Pakistan': 'PAK', 'Philippines': 'PHL', 'Singapore': 'SGP', 'Sri Lanka'

In [26]:
for k, v in d_country_code.items():
    stores_df.loc[(stores_df.country == k), 'iso_alpha'] = v

stores_df.dropna(subset=['iso_alpha' ], inplace=True) 

stores_df

,year,country,migration_no,iso_alpha
29,1959,Fiji,23,FJI
30,1959,Fiji,84,FJI
31,1960,Fiji,87,FJI
32,1961,Fiji,54,FJI
33,1962,Fiji,46,FJI
...,...,...,...,...
2100,1970,South Africa,1593,ZAF
2101,1971,South Africa,1347,ZAF
2102,1972,South Africa,1190,ZAF
2103,1973,South Africa,1154,ZAF


In [27]:
import plotly.express as px
gapminder = px.data.gapminder()
px.choropleth(stores_df,               
              locations="iso_alpha",               
              color="migration_no",
              hover_name="country",  
              animation_frame="year",    
              color_continuous_scale='Plasma',  
              height=600             
)

